### 1. Implemented a regularised ridge regression model trained with SGD to create a linear predictive model


Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

and the update equation for $\mathbf{w}_{\text{new}}$ using gradient descent:

\begin{align*}
   \mathbf{w}_{\text{new}} & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

In [1]:
#Imports
import urllib.request
import zipfile
import pandas as pd 
import numpy as np

#### Acquiring Air Quality Data

In [2]:
#Getting Airquality data
doc = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doc, pat_sav)

#Extracting data
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')
    
#importing data
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

In [3]:
#display a part of the data
air_quality_full.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


#### Sanitizing the data and removing empty instances.

In [4]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]
# and the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

In [5]:
np.random.seed(12345)                 # Adding a seed to get consistent results
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

#### Preprocessing - Imputing missing values and standardisation

In [6]:
#Function to impute missing values, standardise the dataset and return both transformed set as well as imputed values
def preprocessing(dataset,values_in={}):
    values = {}
    dataset.replace(to_replace=-200, value=np.NaN, inplace=True)
    for col in dataset.columns:
        if col in values_in:
            col_mean = values_in[col]['mean']
            col_std_dev = values_in[col]['std_dev']
        else:
            col_mean = dataset[col].mean()
            col_std_dev = dataset[col].std()
            
        dataset[col] = dataset[col].fillna(col_mean)
        dataset[col]= (dataset[col]-col_mean)/(col_std_dev)
        values[col] = {'mean':col_mean,'std_dev':col_std_dev}
    return dataset, values

#Preprocessing Training Data and saving imputed values
data_training, values = preprocessing(data_training_unproc.copy())

#### Seperating the data into Design Matrix and Targets 

In [7]:
#Function to seperate data into Targets y and Design Matrix X
def seperate(dataset):
    y = np.array(dataset.get('CO(GT)')).reshape(-1,1)
    tmp_data = dataset.drop(columns='CO(GT)')
    tmp_array = tmp_data.to_numpy()
    X = np.hstack((np.ones_like(y), tmp_array))
    return X,y

#Seperating training data and targets
XTrain, yTrain = seperate(data_training.copy())

#### Implementing Ridge Regression using Minibatch Gradient Descent and finding the optimal hyperparameters using Gridsearch

In [8]:
#standardise the validation data using the means and standard deviations computed from the training data.
data_val, _ = preprocessing(data_val_unproc.copy(),values.copy())
XVal, yVal = seperate(data_val.copy())

In [9]:
#Create a grid of values for the parameters 𝛾 and 𝜂 using np.logspace and a grid of values for 𝑆 using np.linspace.
learn_rates = list(np.logspace(start =-2, stop = -1, num = 5)) 
regularization_params = list(np.logspace(start = -4, stop = -1, num = 5))
datapoints = list(np.linspace(start = 16, stop = 80, num = 5))

In [10]:
#Fucntion to Create batches from input data for Minibatch Gradient Descent
def create_batch(XTrain,yTrain,data_size):
    mini_batches = []
    i = 0
    data = np.hstack((XTrain,yTrain))
    batches = data.shape[0] // data_size
    for i in range(int(batches)):
        mini_batch = data[i*data_size:(i+1)*data_size, :]
        x = mini_batch[:, :-1]
        y = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((x, y))
    if data.shape[0] % data_size != 0:
        mini_batch = data[-data_size:]
        x = mini_batch[:, :-1]
        y = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((x, y))
    return mini_batches

#fucntion to calculate RMSE
def calc_rmse(XVal,w,yVal):
    y_hat = np.dot(XVal,w)
    rmse = np.sqrt(((yVal-y_hat)**2).mean())
    return rmse

#Function to Perform Minibatch Gradient Descent and calculate RMSE
def mb_gradientdescent(XTrain,yTrain,learn_rate,regularization_param,data_size,XVal,yVal):
    w = np.ones((XTrain.shape[1],1))
    max_iterations = 200
    for iteration in range(max_iterations):
        mini_batches = create_batch(XTrain,yTrain,data_size)
        for batch in mini_batches:
            x, y = batch
            w_old = w
            f = np.dot(x,w_old)
            loss = y - f
            summ = np.dot(x.T,loss)
            gradient = (2*learn_rate*summ)/data_size
            temp = 1-(regularization_param*learn_rate)
            w = (temp*w_old) + gradient
    
    rmse = calc_rmse(XVal,w,yVal)
    return learn_rate,regularization_param,data_size,rmse

lowest = {}  #Initialize dictionary Variable to store the lowest values.
rmse_prev = 100 

#Computing the Value of w for over each possible permutation of the three given hyperparameters 
for ds in datapoints:
    for rp in regularization_params:
        for lr in learn_rates:
            #Perform GD and Calculate RMSE
            learn_rate,regularization_param,data_size,rmse = mb_gradientdescent(
                                                                    XTrain,
                                                                    yTrain,
                                                                    lr,
                                                                    rp,
                                                                    int(ds),
                                                                    XVal,
                                                                    yVal
                                                                )
            #Choose the values of 𝛾, 𝜂 and 𝑆 that lead to the lowest RMSE and save them.
            if rmse < rmse_prev:
                lowest = {
                    'learn_rate':learn_rate,
                    'regularization_param':regularization_param,
                    'batch_size':data_size,
                    'rmse':rmse}
                
                rmse_prev = rmse
                
#print the lowest parameters
print(lowest)

{'learn_rate': 0.05623413251903491, 'regularization_param': 0.0001, 'batch_size': 48, 'rmse': 0.32331021437771945}


#### Training the model using the best hyperparameters and predict over test data 

In [11]:
#Put together the original training and validation dataset
new_data_train = pd.concat([data_training_unproc, data_val_unproc])

#for each feature, impute the missing values with the mean values of the non-missing values 
#stardardise the new training set
process_data_train, train_values = preprocessing(new_data_train.copy())
XnTrain, ynTrain = seperate(process_data_train.copy())

In [12]:
#Preprocess the test data by imputing the missing data and standardising it
data_test, _ = preprocessing(data_test_unproc.copy(),train_values)
XTest, yTest = seperate(data_test.copy())

#Use the best values of 𝛾, 𝜂 and 𝑆 found in the validation set and train a new regularised linear model with stochastic gradient descent
_,_,_,rmse = mb_gradientdescent(XnTrain,ynTrain,lowest['learn_rate'],lowest['regularization_param'],lowest['batch_size'],XTest,yTest)

#Report the RMSE over the test data
print(f'The RMSE over test data is : {rmse}')

The RMSE over test data is : 0.34308048829970467


### 2. Training a random forest for regression over the air quality dataset

We now use random forests to predict air quality. The tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. 

In [13]:
#imports
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#### Preprocessing the data

In [14]:
#Employ the SimpleImputer method in Scikit-learn to impute the missing values in each column
impute = SimpleImputer(missing_values=-200, strategy='mean')
#Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the StandardScaler method
scale = StandardScaler()

#Create a Pipeline that you can use to preprocess the data by filling missing values and then standardising the features
estimators = [('impute', SimpleImputer(missing_values=-200, strategy='mean')), ('scale', StandardScaler())]
pipe = Pipeline(estimators)

In [15]:
#Create a Function to seperate dataset into data and targets
def seperate2(dataset):
    if type(dataset) is np.ndarray:
        y = dataset[:,0].reshape(-1,1)
        X = dataset[:,1:]
    else:
        y = dataset.get('CO(GT)')
        tmp_data = dataset.drop(columns='CO(GT)')
        X  = tmp_data
    return X,y

#Seperate Training and Validation Data into data and targets
X_Train_rf, y_Train_rf = seperate2(data_training_unproc.copy().to_numpy())
X_Val_rf, y_Val_rf = seperate2(data_val_unproc.copy().to_numpy())

# fit the training data  
pipe.fit(X_Train_rf.copy(),y_Train_rf.copy())

#transform the validation data
X_TTrain = pipe.transform(X_Train_rf.copy())
X_TVal = pipe.transform(X_Val_rf.copy())

#### Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor

In [16]:
#comine data and targets of training and validation dataset
X = np.vstack((X_TTrain,X_TVal))
y = np.hstack((y_Train_rf.ravel(),y_Val_rf.ravel()))

#generate split index of train and validation data for combined data 
split_index = (np.append(np.full((X_TTrain.shape[0],), -1),np.full((X_TVal.shape[0],), 0))).reshape(-1,1)

#Use PredefinedSplit to tell the cross validator which instances to use for training and which ones for validation
split = PredefinedSplit(test_fold = split_index)

#Create a grid of five values for each parameter                      
n_estimators =  list(np.linspace(start = 100, stop = 1000,num = 5)) 
n_estimators =  [int(_) for _ in n_estimators]
max_features = list(np.linspace(start = 3, stop = 6, num = 3))
max_features =  [int(_) for _ in max_features]
max_samples = list(np.logspace(start =-1, stop = -0.01, num = 5))

#create the parameter grid
param_grid = {
    'n_estimators':n_estimators,
    'max_features':max_features,
    'max_samples':max_samples
}

#use GridSearchCV to perform Cross-Validation over the predefined split to find the best parameters
gridsearch = GridSearchCV(
    RandomForestRegressor(),
    param_grid=param_grid,
    cv=split
)

gridsearch.fit(X,y)
#print the best generated parameters
print(f'The best parameters are {gridsearch.best_params_}')

The best parameters are {'max_features': 3, 'max_samples': 0.9772372209558107, 'n_estimators': 325}


#### Train a new model using the best parameters over the whole training data and report the prediction over the test data

In [17]:
#combine training and validation data
unify = pd.concat([data_training_unproc, data_val_unproc])

#seperate the combined dataset into data targets 
X_final, y_final = seperate2(unify.copy())
X_test_rf, y_test_rf = seperate2(data_test_unproc.copy())

#Create and apply a new preprocessing pipeline for imputing and standardising the data
estimators = [('impute', SimpleImputer(missing_values=-200, strategy='mean')), ('scale', StandardScaler())]
pipe2 = Pipeline(estimators)

#fit the pipeline on whole training datasete
pipe2.fit(X_final.copy(), y_final.copy()) 

#use the pipeline to transform the whole trianing and testing dataset
X_Tfinal = pipe.transform(X_final.copy())
X_Ttest = pipe.transform(X_test_rf.copy())

#Fit a random forest regression model to the training data using the best parameters found in Question 2.c
rf = RandomForestRegressor(max_features=gridsearch.best_params_['max_features'], max_samples=gridsearch.best_params_['max_samples'], n_estimators=gridsearch.best_params_['n_estimators'])
rf.fit(X_Tfinal,y_final)

#Compute the RMSE over the test data and report the result
predict = rf.predict(X_Ttest)
rmse = mean_squared_error(y_test_rf,predict, squared=False)
print(f'The RMSE over test data is : {rmse}')

The RMSE over test data is : 0.4077750056194089
